In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
import cv2
import yaml
from sklearn.model_selection import train_test_split

import os

# Set the paths for KITTI dataset and YOLO output
kitti_base_path = 'Kitti_dataset/Training'
yolo_base_path = 'Yolo_annot_dataset'

# Define paths to images and labels
images_path = os.path.join(kitti_base_path, 'training_image')
labels_path = os.path.join(kitti_base_path, 'train_annot')
yolo_images_path = os.path.join(yolo_base_path, 'images')
yolo_labels_path = os.path.join(yolo_base_path, 'labels')

# Print paths for verification
print("Paths set up successfully:")
print(f"KITTI images path: {images_path}")
print(f"KITTI labels path: {labels_path}")
print(f"YOLO images path: {yolo_images_path}")
print(f"YOLO labels path: {yolo_labels_path}")


Paths set up successfully:
KITTI images path: Kitti_dataset/Training\training_image
KITTI labels path: Kitti_dataset/Training\train_annot
YOLO images path: Yolo_annot_dataset\images
YOLO labels path: Yolo_annot_dataset\labels


In [ ]:
# Create YOLO directories
os.makedirs(yolo_images_path, exist_ok=True)
os.makedirs(yolo_labels_path, exist_ok=True)

In [ ]:
kitti_classes = ['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc']


In [ ]:
def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)


In [ ]:
# Process each label file
for label_file in os.listdir(labels_path):
    if not label_file.endswith('.txt'):
        continue
    image_file = label_file.replace('.txt', '.png')

    image = cv2.imread(os.path.join(images_path, image_file))
    h, w, _ = image.shape

    with open(os.path.join(labels_path, label_file), 'r') as lf:
        lines = lf.readlines()
    yolo_labels = []
    for line in lines:
        elements = line.strip().split(' ')
        class_id = elements[0]
        if class_id in kitti_classes:
            class_id = kitti_classes.index(class_id)  # In KITTI, the first element is the class
            xmin, ymin, xmax, ymax = map(float, elements[4:8])
            bbox = convert_bbox((w, h), (xmin, xmax, ymin, ymax))
            yolo_labels.append(f"{class_id} {' '.join(map(str, bbox))}\n")

    with open(os.path.join(yolo_labels_path, label_file), 'w') as yf:
        yf.writelines(yolo_labels)
    # Copy image to YOLO directory
    shutil.copy(os.path.join(images_path, image_file), yolo_images_path)


In [ ]:
# Check if the images directory exists
if not os.path.exists(yolo_images_path):
    print(f"Directory does not exist: {yolo_images_path}")
else:
    print(f"Directory exists: {yolo_images_path}")

# Now list all images
all_images = [f for f in os.listdir(yolo_images_path) if f.endswith('.png')]
train_images, test_images = train_test_split(all_images, test_size=0.2, random_state=42)
val_images, test_images = train_test_split(test_images, test_size=0.5, random_state=42)

Directory exists: Yolo_annot_dataset\images


In [ ]:
def move_files(file_list, dest_dir):
    os.makedirs(os.path.join(dest_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(dest_dir, 'labels'), exist_ok=True)
    for file_name in file_list:
        shutil.move(os.path.join(yolo_images_path, file_name), os.path.join(dest_dir, 'images', file_name))
        label_file = file_name.replace('.png', '.txt')
        shutil.move(os.path.join(yolo_labels_path, label_file), os.path.join(dest_dir, 'labels', label_file))

move_files(train_images, os.path.join(yolo_base_path, 'train'))
move_files(val_images, os.path.join(yolo_base_path, 'val'))
move_files(test_images, os.path.join(yolo_base_path, 'test'))


In [ ]:
data = {
    'train': '/kaggle/working/yolo/train/images',
    'val': '/kaggle/working/yolo/val/images',
    'test': '/kaggle/working/yolo/test/images',
    'nc': 8,
    'names': ['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc']
}

with open('Yolo_annot_dataset/data.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)
